#### deriving RV uncertainties using the Cramer-Rao bound

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import wobble
from tqdm import tqdm

/usr/local/Cellar/python3/3.6.0/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
starname = '51peg'
K_star = 0
K_t = 3    
star_reg_file = 'wobble/regularization/{0}_star_K{1}.hdf5'.format(starname, K_star)
tellurics_reg_file = 'wobble/regularization/{0}_t_K{1}.hdf5'.format(starname, K_t)

In [3]:
data = wobble.Data(starname+'_e2ds.hdf5', filepath='../data/', orders=[30])

In [4]:
results = wobble.Results(data=data)

In [5]:
model = wobble.Model(data, results, 0)
model.add_star('star', variable_bases=K_star, 
                regularization_par_file=star_reg_file)
model.add_telluric('tellurics', rvs_fixed=True, variable_bases=K_t, 
                    regularization_par_file=tellurics_reg_file)

Regularization parameter file wobble/regularization/51peg_star_K0.hdf5 not recognized; using keywords instead.
Regularization parameter file wobble/regularization/51peg_t_K3.hdf5 not recognized; using keywords instead.


In [6]:
wobble.optimize_order(model, niter=80, save_history=True, 
                      basename='../results/plots_{0}/history'.format(starname))

100%|██████████| 80/80 [00:19<00:00,  4.20it/s]


<Figure size 432x288 with 0 Axes>

In [7]:
n = 0 # epoch
r = 0 # order idx

In [8]:
model.components[0].synth[n]

<tf.Tensor 'strided_slice_168:0' shape=(4096,) dtype=float64>

In [9]:
model.components[0].rvs

<tf.Variable 'rvs_star:0' shape=(91,) dtype=float64_ref>

In [12]:
session = wobble.get_session()
M = 2048 #len(data.xs[r][n])

dsynth_dv = [session.run(tf.gradients(model.components[0].synth[n][m], 
                                      model.components[0].rvs))[0][n] for m in tqdm(range(M))]

 20%|██        | 418/2048 [07:13<28:11,  1.04s/it]

KeyboardInterrupt: 

In [11]:
crlb = np.sum(dsynth_dv * data.ivars[r][n][:M] * dsynth_dv)
print(1./np.sqrt(crlb))

6.78358776810144


#### deriving RV uncertainties using the Hessian

In [19]:
dnll_dv = tf.gradients(model.nll, model.components[0].rvs)
best_rv = results.star_rvs[r][n]
current = session.run(dnll_dv[n], feed_dict={model.components[0].rvs[n]:best_rv})
tiny = 100.
hessian = (session.run(dnll_dv[n], feed_dict={model.components[0].rvs[n]:best_rv+tiny}) 
           - current)/tiny

In [20]:
hessian

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.])

In [18]:
current

array([-1.29501219, -1.31234992, -1.70108225, -1.20127702, -1.74705781,
       -0.65362017, -0.32896344, -0.54877724, -0.53440751, -0.57135449,
       -0.80785374, -0.44906273, -0.79987442, -0.5259963 , -0.72445399,
       -0.39236502, -0.78200905, -0.64688869, -0.45338347, -0.39344526,
       -0.17199331, -0.3672373 , -0.63948043, -0.76085476, -0.64625487,
       -0.5248757 , -0.37946589, -0.7260073 , -0.65740683, -0.97675626,
       -0.68548231, -0.60995971, -0.559902  , -0.39925021, -0.8024303 ,
        0.06499479,  0.16528742,  0.11849824,  0.07036738,  0.48388063,
        0.56799801,  0.91134978,  0.83688227,  0.61566179,  0.66389593,
        0.67325085,  0.42713236,  0.30456814,  0.26287009,  0.18306345,
        0.32925854,  0.46942686, -0.15789846, -0.11546799, -0.09676384,
        0.13125438,  0.45567952,  0.02382348,  0.01108982,  0.11727961,
        0.91542757,  0.15522488,  0.16152347,  0.22740634,  0.3453133 ,
        0.64911191,  0.6456514 ,  0.87180115,  1.0966158 ,  1.26